In [ ]:
from pyspark.sql.functions import col,av
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("HogwartsAnalysis").getOrCreate()


In [10]:
data = [
    ("Gryffindor", 1, 80, "Harry Potter"),
    ("Slytherin", 1, 60, "Draco Malfoy"),
    ("Ravenclaw", 1, 45, "Luna Lovegood"),
    ("Hufflepuff", 1, 30, "Cedric Diggory"),
    ("Gryffindor", 2, 90, "Hermione Granger"),
    ("Slytherin", 2, 70, "Pansy Parkinson"),
    ("Ravenclaw", 2, 55, "Cho Chang"),
    ("Hufflepuff", 2, 65, "Hannah Abbott"),
    ("Gryffindor", 3, 20, "Ron Weasley"),
    ("Slytherin", 3, 85, "Blaise Zabini")
]

In [11]:
columns = ["house", "year", "points", "student"]
df = spark.createDataFrame(data, columns)
df.show()

+----------+----+------+----------------+
|     house|year|points|         student|
+----------+----+------+----------------+
|Gryffindor|   1|    80|    Harry Potter|
| Slytherin|   1|    60|    Draco Malfoy|
| Ravenclaw|   1|    45|   Luna Lovegood|
|Hufflepuff|   1|    30|  Cedric Diggory|
|Gryffindor|   2|    90|Hermione Granger|
| Slytherin|   2|    70| Pansy Parkinson|
| Ravenclaw|   2|    55|       Cho Chang|
|Hufflepuff|   2|    65|   Hannah Abbott|
|Gryffindor|   3|    20|     Ron Weasley|
| Slytherin|   3|    85|   Blaise Zabini|
+----------+----+------+----------------+



In [30]:
new_df=df.filter(col("points")>50).groupBy("house","year").agg(sum("points").alias("total_points"))
final_df=new_df.orderBy(col("year"),col("total_points").desc())
final_df.show()

+----------+----+------------+
|     house|year|total_points|
+----------+----+------------+
|Gryffindor|   1|          80|
| Slytherin|   1|          60|
|Gryffindor|   2|          90|
| Slytherin|   2|          70|
|Hufflepuff|   2|          65|
| Ravenclaw|   2|          55|
| Slytherin|   3|          85|
+----------+----+------------+



In [64]:
data = [
    ("Aragorn", "Human", 10, 2, "Helms Deep"),
    ("Legolas", "Elf", 15, 0, "Helms Deep"),
    ("Gimli", "Dwarf", 8, 3, "Helms Deep"),
    ("Frodo", "Hobbit", 2, 1, "Moria"),
    ("Sam", "Hobbit", 4, 2, "Moria"),
    ("Gandalf", "Wizard", 12, 1, "Moria"),
    ("Boromir", "Human", 7, 4, "Amon Hen"),
    ("Legolas", "Elf", 20, 0, "Amon Hen"),
    ("Aragorn", "Human", 9, 2, "Amon Hen")
]


In [65]:
columns = ["name", "race", "enemies_defeated", "injuries", "battle"]
df = spark.createDataFrame(data, columns)
df.show()


+-------+------+----------------+--------+----------+
|   name|  race|enemies_defeated|injuries|    battle|
+-------+------+----------------+--------+----------+
|Aragorn| Human|              10|       2|Helms Deep|
|Legolas|   Elf|              15|       0|Helms Deep|
|  Gimli| Dwarf|               8|       3|Helms Deep|
|  Frodo|Hobbit|               2|       1|     Moria|
|    Sam|Hobbit|               4|       2|     Moria|
|Gandalf|Wizard|              12|       1|     Moria|
|Boromir| Human|               7|       4|  Amon Hen|
|Legolas|   Elf|              20|       0|  Amon Hen|
|Aragorn| Human|               9|       2|  Amon Hen|
+-------+------+----------------+--------+----------+



In [73]:
from pyspark.sql.functions import *
new_df=df.select("name","race","enemies_defeated")
new_filtered=new_df.filter(col("enemies_defeated")>5).select("name","race","enemies_defeated")
grouped_df=new_filtered.groupBy("race").agg(avg(col("enemies_defeated")).alias("avg_ed"))
final_df=grouped_df.orderBy(col("avg_ed").desc())
final_df.show()

+------+-----------------+
|  race|           avg_ed|
+------+-----------------+
|   Elf|             17.5|
|Wizard|             12.0|
| Human|8.666666666666666|
| Dwarf|              8.0|
+------+-----------------+



In [52]:
data = [
    ("SUB-01", "Pacific Strike", 5, "Success"),
    ("SUB-02", "Atlantic Surge", 2, "Failure"),
    ("SUB-01", "Arctic Blitz", 4, "Success"),
    ("SUB-03", "Indian Ocean", 6, "Success"),
    ("SUB-02", "Pacific Strike", 3, "Success"),
    ("SUB-01", "Coral Sea", 7, "Success"),
    ("SUB-03", "Arctic Blitz", 1, "Failure"),
    ("SUB-02", "Bering Strait", 5, "Success")
]

In [53]:
columns = ["submarine_id", "mission_name", "warheads_launched", "status"]
df = spark.createDataFrame(data, columns)
df.show()

+------------+--------------+-----------------+-------+
|submarine_id|  mission_name|warheads_launched| status|
+------------+--------------+-----------------+-------+
|      SUB-01|Pacific Strike|                5|Success|
|      SUB-02|Atlantic Surge|                2|Failure|
|      SUB-01|  Arctic Blitz|                4|Success|
|      SUB-03|  Indian Ocean|                6|Success|
|      SUB-02|Pacific Strike|                3|Success|
|      SUB-01|     Coral Sea|                7|Success|
|      SUB-03|  Arctic Blitz|                1|Failure|
|      SUB-02| Bering Strait|                5|Success|
+------------+--------------+-----------------+-------+



In [56]:
filt_df = df.filter((col("status") == "Success") & (col("warheads_launched") > 3)).select("submarine_id", "mission_name", "warheads_launched")
filt_df.show()

+------------+--------------+-----------------+
|submarine_id|  mission_name|warheads_launched|
+------------+--------------+-----------------+
|      SUB-01|Pacific Strike|                5|
|      SUB-01|  Arctic Blitz|                4|
|      SUB-03|  Indian Ocean|                6|
|      SUB-01|     Coral Sea|                7|
|      SUB-02| Bering Strait|                5|
+------------+--------------+-----------------+



In [62]:
grouped_df=filt_df.groupBy("submarine_id").agg(count(col("mission_name")).alias("count_of_missions"),sum(col("warheads_launched")).alias("total-warheads"))
final_df=grouped_df.orderBy(col("total-warheads").desc())
final_df.show()

+------------+-----------------+--------------+
|submarine_id|count_of_missions|total-warheads|
+------------+-----------------+--------------+
|      SUB-01|                3|            16|
|      SUB-03|                1|             6|
|      SUB-02|                1|             5|
+------------+-----------------+--------------+

